In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 7860 files belonging to 6 classes.


In [4]:
class_name = ['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [5]:
def get_partition(ds, train_split = 0.8, val_split = 0.1, test_split = 0.1, shuffle = True, shuffle_size = 10000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).take(val_size)

    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_partition(dataset)

In [7]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [8]:
data_scale = keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    keras.layers.experimental.preprocessing.Rescaling(1. / 255)
])

In [9]:
data_augment = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [10]:
train_ds = train_ds.map(
    lambda x, y: (data_augment(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [11]:
import os
import tensorflow as tf

# Path to the directory where models are saved
model_dir = "../plant_models"

# List of model versions (folders or files) in the directory
model_versions = [i for i in os.listdir(model_dir) if i.isdigit()]

# Get the latest model version
if model_versions:
    latest_version = max([int(i) for i in model_versions])
    model_path = os.path.join(model_dir, str(latest_version))
else:
    raise ValueError("No models found in the directory.")

# Load the model
model = tf.keras.models.load_model(f"../plant_models/{latest_version}")

print(f"Loaded model from version {latest_version}")

Loaded model from version 3


In [ ]:
history = model.fit(train_ds, epochs = 70, batch_size = 32, verbose = 1, validation_data = val_ds)

Epoch 1/70
196/196 [==============================] - 58s 253ms/step - loss: 0.1193 - accuracy: 0.9584 - val_loss: 0.1756 - val_accuracy: 0.9479
Epoch 2/70
196/196 [==============================] - 40s 204ms/step - loss: 0.0883 - accuracy: 0.9681 - val_loss: 0.0813 - val_accuracy: 0.9753
Epoch 3/70
196/196 [==============================] - 41s 209ms/step - loss: 0.0904 - accuracy: 0.9678 - val_loss: 0.0818 - val_accuracy: 0.9766
Epoch 4/70
196/196 [==============================] - 45s 228ms/step - loss: 0.0711 - accuracy: 0.9732 - val_loss: 0.0668 - val_accuracy: 0.9779
Epoch 5/70
196/196 [==============================] - 46s 237ms/step - loss: 0.0503 - accuracy: 0.9833 - val_loss: 0.0582 - val_accuracy: 0.9740
Epoch 6/70
196/196 [==============================] - 47s 239ms/step - loss: 0.0541 - accuracy: 0.9825 - val_loss: 0.0893 - val_accuracy: 0.9648
Epoch 7/70
196/196 [==============================] - 47s 239ms/step - loss: 0.0539 - accuracy: 0.9820 - val_loss: 0.0410 - val_ac

In [ ]:
import os

if not os.path.exists("../plant_models"):
    os.makedirs("../plant_models")

model_versions = [i for i in os.listdir("../plant_models") if i.isdigit()]

model_version = max([int(i) for i in model_versions] + [0]) + 1

model.save(f"../plant_models/{model_version}")